## Business Brochure for a Company

Generation of a Brochure for a company to be used for prospective clients, investors and potential recruits.


In [1]:
# library imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()


API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url # stores the URL provided
        response = requests.get(url, headers=headers)  # sends an HTTP get request to the URL
        self.body = response.content  # stores the raw HTML content of the web
        soup = BeautifulSoup(self.body, 'html.parser')  # parsing of the html content
        self.title = soup.title.string if soup.title else "No title found" # extracts the webpage title if available
        if soup.body:   # if webpage has a body 
            for irrelevant in soup.body(["script", "style", "img", "input"]): # removes unnecessary elements
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True) # all the other elements are separated by line breaks
        else:
            self.text = ""     # if there is no body, self.text is set to empty string
        links = [link.get('href') for link in soup.find_all('a')]  # in web scraping, a is a tag useful for finding certain elements
        self.links = [link for link in links if link]  # in this case it helps finding links (<a href ...>)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"  # returns a formatted string containing the title and textual content of the webpage.

In [4]:
ed = Website("https://edwarddonner.com") # the class is initialized with this web links and all the operations defined above are executed
ed.links # all the links are extracted here

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

Sidenote: there is a technique called "Structured Outputs" in which we require the model to respond according to a specific task.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Let's scrape Hugging's Face website

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen2.5-Omni-7B',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/manycore-research/SpatialLM-Llama-1B',
 '/ds4sd/SmolDocling-256M-preview',
 '/ByteDance/InfiniteYou',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Qwen/Qwen2.5-Omni-7B-Demo',
 '/spaces',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/PixelAI-Team/TalkBody4D',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen2.5-Omni-7B
Updated
2 days ago
•
61.8k
•
1.04k
deepseek-ai/DeepSeek-V3-0324
Updated
6 days ago
•
101k
•
2.21k
manycore-research/SpatialLM-Llama-1B
Updated
12 days ago
•
13.2k
•
857
ds4sd/SmolDocling-256M-preview
Updated
10 days ago
•
60.7k
•

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen2.5-Omni-7B\nUpdated\n2 days ago\n•\n61.8k\n•\n1.04k\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n6 days ago\n•\n101k\n•\n2.21k\nmanycore-research/SpatialLM-Llama-1B\nUpdated\n12 days ago\n•\n13.2k\n•\n857\nds4sd/SmolDocling-256M-preview\nUpdated\n10 days ago\n•\n60.7k\n•\n1.11k\nByteDance/InfiniteYou\nUpdated\nabout 7 hours ago\n•\n516\nBrowse 1M+ models\nSpaces\nRunning\n1.44k\n1.44k\nDe

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content  #choices 0 means we'll receive one variation of the request
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Hugging Face: The AI Community Building the Future

## About Us
Hugging Face is redefining the landscape of Artificial Intelligence (AI) and Machine Learning (ML). We are a collaborative platform where the AI community comes together to share, create, and innovate using an extensive array of models, datasets, and applications. Our mission is to democratize AI, making it accessible for everyone to harness the power of technology in various domains.

## Our Offerings
- **Models**: Access over **1 million models** tailored for numerous applications, helping users to implement AI solutions effortlessly.
- **Datasets**: Explore a rich library of **250,000+ datasets** designed to support diverse ML tasks.
- **Spaces**: Engage with our community to create and share applications in AI, with seamless collaboration at the core.
- **Enterprise Solutions**: Advanced tools for organizations, providing AI capabilities with industry-standard security and support.

## Company Culture
At Hugging Face, we foster an open and inclusive environment that emphasizes collaboration, creativity, and community. Our **209 team members** are passionate about AI and dedicated to building tools that empower users. We believe in the **open-source culture** and actively encourage contributions from developers around the globe to advance ML technology.

## Our Community
More than **50,000 organizations** utilize Hugging Face for their AI needs, including tech giants like Amazon, Google, Microsoft, and innovative startups. **AI and ML enthusiasts** are not just users; they are contributors who engage with over **142,000 transformers** models and various AI tools we offer.

## Careers at Hugging Face
We are always looking for talented individuals to join our mission. If you're passionate about AI, open-source projects, and transforming how technology interacts with human creativity, we invite you to explore our **career opportunities**. Join us in our journey to build a future where AI benefits all.

## Join Us!
Whether you're an AI enthusiast, a business looking for innovative solutions, or a potential team member, we welcome you to Hugging Face. Together, let's build the future of machine learning and elevate the community.

**Explore More**:  
[Models](https://huggingface.co/models) | [Datasets](https://huggingface.co/datasets) | [Community Spaces](https://huggingface.co/spaces) | [Careers](https://huggingface.co/jobs)

Connect with us on [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/hugging-face) | [GitHub](https://github.com/huggingface)
```


## A minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## Company Overview
**Hugging Face** is at the forefront of the artificial intelligence revolution, creating a collaborative platform for developers, researchers, and businesses. As part of a vibrant AI community, we aim to build the future of machine learning with shared models, datasets, and applications.

### What We Offer
- **1 Million+ Models**: Explore an extensive library of pre-trained models for various tasks including text, image, and audio processing.
- **250k+ Datasets**: Access diverse datasets needed for machine learning and AI development.
- **Spaces**: Utilize interactive applications that facilitate experimentation and immediate feedback.
- **Enterprise Solutions**: Tailored options for organizations with enterprise-grade security, support, and advanced features.

## Company Culture
At Hugging Face, we pride ourselves on fostering a **community-oriented culture** that encourages collaboration and innovation. Our open-source commitment illustrates our dedication to democratizing AI. We believe in transparency, inclusion, and the collective power of contributions from the global AI community.

### Core Values:
- **Collaboration**: We work together to solve challenges in AI.
- **Open Source**: Our tools are developed openly and freely available.
- **Mission-driven**: Focused on creating a positive impact through technology.

## Our Community
More than **50,000 organizations** trust Hugging Face, including industry leaders like **Meta**, **Amazon**, **Google**, and **Microsoft**. We empower them by providing the tools needed to advance their AI initiatives rapidly.

### Join the Community
Whether you are a researcher, an enterprise, or a developer, participating in the Hugging Face ecosystem can accelerate your projects and ideas.

## Careers at Hugging Face
Hugging Face offers exciting career opportunities for those passionate about AI and machine learning. Join our diverse team of innovators who are reshaping the AI landscape. We are on the lookout for talent in various roles, from engineering and research to marketing and community support.

### Current Openings
Explore current job opportunities on our [Careers Page](https://huggingface.co/jobs) to find your place in our mission.

## Get Started With Us
To learn more about our offerings or to start collaborating, visit our official site: [Hugging Face](https://huggingface.co). You can also explore our vast resources, engage with our community, or sign up for one of our many services today!

---

**Contact Us**:  
Website: [huggingface.co](https://huggingface.co)  
Social: [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface) | [Discord](https://discord.gg/huggingface)

